In [16]:
import pandas as pd

In [17]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

----

In [35]:
airports_df = pd.read_csv('../data/airport_dataset/reference_w_airports.csv')
sports_df = pd.read_csv('../data/sports_dataset/reference_w_sports.csv')
population_df = pd.read_csv('../data/population_dataset/reference_w_population.csv')
meteorite_df = pd.read_csv('../data/meteorite_dataset/reference_w_meteorite.csv')

In [36]:
ref_df = pd.read_csv('../data/reference_w_loc.csv')

#### Sanity check: make sure all DFs have same number of rows

In [37]:
len(ref_df.index) == len(airports_df.index) == len(sports_df.index) == len(population_df.index) == len(meteorite_df.index)

False

In [38]:
ref_df.shape

(38511, 12)

In [39]:
airports_df.shape

(38511, 20)

In [40]:
sports_df.shape

(38511, 17)

In [41]:
population_df.shape

(38439, 14)

In [42]:
meteorite_df.shape

(38511, 17)

### Merge ref with airports

In [43]:
airports_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'closest_LARGE_airport_name',
       u'closest_LARGE_airport_distance', u'closest_MEDIUM_airport_name',
       u'closest_MEDIUM_airport_distance', u'closest_SMALL_airport_name',
       u'closest_SMALL_airport_distance', u'closest_CLOSED_airport_name',
       u'closest_CLOSED_airport_distance'],
      dtype='object')

In [45]:
airports_subset_df = airports_df[['id','geocoded_latitude','geocoded_longitude','closest_LARGE_airport_distance', 'closest_MEDIUM_airport_distance','closest_SMALL_airport_distance']]

In [48]:
airports_subset_df.columns.values[1] = 'airport_lat'
airports_subset_df.columns.values[2] = 'airport_lon'

In [49]:
airports_subset_df.head(1)

,id,airport_lat,airport_lon,closest_LARGE_airport_distance,closest_MEDIUM_airport_distance,closest_SMALL_airport_distance
0,0,41.661256,-91.529911,12.498008,6.616011,0.683006


In [50]:
ref_airports = ref_df.merge(airports_subset_df, on='id')

### Merge ref_airports with sports

In [51]:
sports_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'closest_metro_name',
       u'closest_metro_distance', u'closest_metro_pop', u'closest_metro_m4',
       u'closest_metro_m6'],
      dtype='object')

In [52]:
sports_df.closest_metro_pop = sports_df.closest_metro_pop.str.replace(",","").astype(float)

In [54]:
sports_subset_df = sports_df[['id', 'geocoded_latitude', 'geocoded_longitude', 'closest_metro_distance', 'closest_metro_pop', 'closest_metro_m4', 'closest_metro_m6']]

In [55]:
sports_subset_df.columns.values[1] = 'sports_metro_lat'
sports_subset_df.columns.values[2] = 'sports_metro_lon'

In [56]:
ref_airports_sports = ref_airports.merge(sports_subset_df, on='id')

### Merge ref_airports_sports with population

In [57]:
population_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'year', u'population'],
      dtype='object')

In [58]:
pop_df = population_df[['id','population']]

In [59]:
pop_df.columns

Index([u'id', u'population'], dtype='object')

In [60]:
ref_airports_sports_population = ref_airports_sports.merge(pop_df, on='id')

In [65]:
ref_airports_sports_population.shape

(38439, 24)

### Merge ref_airports_sports_population with meteorite

In [66]:
meteorite_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'meteor_name', u'distance_to_meteor',
       u'reason', u'metorite_lat', u'metorite_long'],
      dtype='object')

In [67]:
len(meteorite_df['distance_to_meteor'].notnull())

38511

In [68]:
meteorite_df['meteor_sighting'] = meteorite_df['distance_to_meteor'].notnull().astype(int)

In [69]:
meteorite_subset_df = meteorite_df[['id', 'geocoded_latitude', 'geocoded_longitude','meteor_sighting']]

In [70]:
meteorite_subset_df.columns.values[1] = 'meteorite_lat'
meteorite_subset_df.columns.values[2] = 'meteorite_lon'

In [71]:
final_df = ref_airports_sports_population.merge(meteorite_subset_df, on='id')

In [72]:
final_df.shape

(38439, 27)

In [73]:
final_df.sample()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,...,sports_metro_lat,sports_metro_lon,closest_metro_distance,closest_metro_pop,closest_metro_m4,closest_metro_m6,population,meteorite_lat,meteorite_lon,meteor_sighting
1353,1764,Summary : A streak of greenish and white light...,5 seconds,"Frontenac, MO",19970708,light,19970704,38.635844,-90.414979,Frontenac,...,38.635844,-90.414979,11.967999,2195000.0,2.0,2.0,5595211,38.635844,-90.414979,0


In [75]:
final_df.to_csv("ufo_dataset_final.csv" ,index=False)

In [76]:
final_df.to_csv("ufo_dataset_final_with_lat_lon.csv" ,index=False)

In [3]:
# final_df.to_pickle('../data/pickle_files/ufo_dataset_final.pkl')
# final_df = pd.read_pickle('../data/pickle_files/ufo_dataset_final.pkl')

In [77]:
# final_df.to_pickle('../data/pickle_files/ufo_dataset_final_with_lat_lon.pkl')
# final_df = pd.read_pickle('../data/pickle_files/ufo_dataset_final_with_lat_lon.pkl')

In [80]:
list(final_df.columns.values)

['id',
 'description',
 'duration',
 'location',
 'reported_at',
 'shape',
 'sighted_at',
 'geocoded_latitude',
 'geocoded_longitude',
 'city',
 'state',
 'country',
 'airport_lat',
 'airport_lon',
 'closest_LARGE_airport_distance',
 'closest_MEDIUM_airport_distance',
 'closest_SMALL_airport_distance',
 'sports_metro_lat',
 'sports_metro_lon',
 'closest_metro_distance',
 'closest_metro_pop',
 'closest_metro_m4',
 'closest_metro_m6',
 'population',
 'meteorite_lat',
 'meteorite_lon',
 'meteor_sighting']

In [81]:
ftrs = ['closest_LARGE_airport_distance', 'closest_MEDIUM_airport_distance', 'closest_SMALL_airport_distance', 'closest_metro_distance', 'closest_metro_pop', 'closest_metro_m4', 'closest_metro_m6', 'population', 'meteor_sighting']

In [82]:
df_only_ftrs = final_df[ftrs]

In [83]:
normalized_final_df=(df_only_ftrs-df_only_ftrs.min())/(df_only_ftrs.max()-df_only_ftrs.min())

In [84]:
normalized_final_df['id'] = final_df['id']
# airport coords
normalized_final_df['airport_lat'] = final_df['airport_lat']
normalized_final_df['airport_lon'] = final_df['airport_lon']
# sports coords
normalized_final_df['sports_metro_lat'] = final_df['sports_metro_lat']
normalized_final_df['sports_metro_lon'] = final_df['sports_metro_lon']
# meteor coords
normalized_final_df['meteorite_lat'] = final_df['meteorite_lat']
normalized_final_df['meteorite_lon'] = final_df['meteorite_lon']

In [85]:
# normalized_final_df.to_csv('normalized_dataset_final.csv',index=False)
normalized_final_df.to_csv('normalized_dataset_final_with_lat_lon.csv',index=False)